In [2]:
import pandas as pd
import numpy as np

from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 90%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 95%; }
</style>
"""))
# pd.set_option('max_colwidth', 100)

In [3]:
# mf = pd.read_csv('buildstock_MF.csv', index_col='Building')
# sfa = pd.read_csv('buildstock_SFA.csv', index_col='Building')

In [4]:
#Change HVAC options from central systems to unit
buildstock_file = 'buildstock_SFA_10000.csv'
mf = pd.read_csv(buildstock_file, index_col='Building')

# for idx in mf.index:
#     row = mf.loc[idx]
#     if row['HVAC System Is Shared'] == 'None':
#         continue
#     fuel_cols = ['Natural Gas']
#     if mf.loc[idx]['HVAC System Shared Natural Gas'] == 'Boiler Baseboards Heating Only':
#         mf.loc[idx, 'HVAC System Heating Natural Gas'] = 'Gas Boiler, 80% AFUE'
#         mf.loc[idx, 'HVAC System Shared Natural Gas'] = 'None'
        
#     elif mf.loc[idx]['HVAC System Shared Natural Gas'] == 'Fan Coil Heating And Cooling':
#         mf.loc[idx, 'HVAC System Heating Natural Gas'] = 'Gas Boiler, 80% AFUE'
#         mf.loc[idx, 'HVAC System Cooling'] = 'AC, SEER 10'
#         mf.loc[idx, 'HVAC System Shared Natural Gas'] = 'None'
        
#     elif mf.loc[idx]['HVAC System Shared Natural Gas'] == 'Fan Coil Cooling Only':
#         mf.loc[idx, 'HVAC System Cooling'] = 'AC, SEER 10'
#         mf.loc[idx, 'HVAC System Shared Natural Gas'] = 'None'
        
#     elif mf.loc[idx]['HVAC System Shared Natural Gas'] == 'PTAC Heating And Cooling':
#         mf.loc[idx, 'HVAC System Heating Natural Gas'] = 'Gas Boiler, 80% AFUE'
#         mf.loc[idx, 'HVAC System Cooling'] = 'AC, SEER 10'
#         mf.loc[idx, 'HVAC System Shared Natural Gas'] = 'None'
        
#     elif mf.loc[idx]['HVAC System Shared Electricity'] == 'Boiler Baseboards Heating Only':
#         mf.loc[idx, 'HVAC System Heating Electricity'] = 'Electric Baseboard'
#         mf.loc[idx, 'HVAC System Shared Electricity'] = 'None'
    
#     elif mf.loc[idx]['HVAC System Shared Electricity'] == 'Fan Coil Cooling Only': 
#         mf.loc[idx, 'HVAC System Cooling'] = 'AC, SEER 10'
#         mf.loc[idx, 'HVAC System Shared Electricity'] = 'None'
        
#     elif mf.loc[idx]['HVAC System Shared Electricity'] == 'Fan Coil Heating And Cooling':
#         mf.loc[idx, 'HVAC System Cooling'] = 'AC, SEER 10'
#         mf.loc[idx, 'HVAC System Heating Electricity'] = 'Electric Baseboard'
#         mf.loc[idx, 'HVAC System Shared Electricity'] = 'None'
    
#     elif mf.loc[idx]['HVAC System Shared Propane'] == 'Boiler Baseboards Heating Only':
#         mf.loc[idx, 'HVAC System Heating Propane'] = 'Propane Boiler, 80% AFUE'
#         mf.loc[idx, 'HVAC System Shared Propane'] = 'None'
        
#     elif mf.loc[idx]['HVAC System Shared Fuel Oil'] == 'Boiler Baseboards Heating Only':
#         mf.loc[idx, 'HVAC System Heating Fuel Oil'] = 'Oil Boiler, 80% AFUE'
#         mf.loc[idx, 'HVAC System Shared Fuel Oil'] = 'None'    

In [6]:
# mf.to_csv('buildstock_SFA.csv')

In [5]:
dirs = ['North', 'Northeast', 'East', 'Southeast', 'South', 'Southwest', 'West', 'Northwest']
inv_dirs = ['South', 'Southwest', 'West', 'Northwest', 'North', 'Northeast', 'East', 'Southeast']
orient_dict = dict(zip(dirs, inv_dirs))

In [6]:
master_df = pd.DataFrame()
buildtype = 'SFA'
df = mf

middle_ct = 0
for idx in df.index:
    bldg = df.loc[idx]
    units = int(bldg[f'Geometry Building Number Units {buildtype}'])
    bstock = pd.DataFrame(columns= df.columns)
    bstock.index.name = 'Building'
    
    if units >= 3 and buildtype == 'SFA':
        mid_mult = units-2
        units = 3
    else:
        mid_mult = 1
    
    if buildtype == 'MF':
        not_type = 'SFA'
        floors = int(bldg['Geometry Stories'])
        num_units_per_floor = units/floors
        front_orient = bldg['Orientation']
        rear_orient = front_orient
        num_middle_z = floors - 2
        if num_units_per_floor%2 != 0: #no rear
            has_rear_units = "No"
            num_middle_x = int(num_units_per_floor - 2)
        else: #rear units
            has_rear_units = "Yes"
            rear_orient = orient_dict[bldg['Orientation']]
            num_middle_x = int((num_units_per_floor/2) - 2)   
        if num_middle_x < 0:
            num_middle_x = 0
        if has_rear_units == 'Yes':
            num_units_front = int(units/2)
            units_add = int(num_units_per_floor/2)
        else:
            num_units_front = int(units)
            units_add = int(num_units_per_floor)
    else:
        not_type = 'MF'
        num_middle_x = units - 2
        has_rear_units = "No"

    #Initial horizontal locations
    h_left = [1]
    if num_middle_x > 0:
        h_mid = list(range(2, 2+int(num_middle_x)))
    else:
        h_mid = []
    if (buildtype=='SFA') or (num_units_front/floors > 1 and buildtype=='MF'):
        h_right = [num_middle_x + 2]
    else:
        h_right = []
        
    #Define horizontal locatons by floor
    if buildtype == 'MF':
        for f in range(2,floors+1):
            h_left += [h_left[-1] + units_add]
            h_mid += list(map(lambda x: x+units_add, h_mid[-num_middle_x:]))
            h_right += list(map(lambda x: x+units_add, h_right[-1:]))

    #LEVELS
    if buildtype == 'MF':
        #..bottom
        if num_units_front/floors > 1:
            lv_bottom = list(range(1,2+num_middle_x+1))
        else:
            lv_bottom = [1]
        #..middle
        if num_middle_z > 0:
            lv_middle = list(range(lv_bottom[-1]+1, lv_bottom[-1]+1 + (units_add*num_middle_z)))
        else:
            lv_middle = []
        #..top
        if floors > 1:
            lv_top = list(range(units_add*(floors-1) + 1, units_add*floors +1))
        else:
            lv_top = []

        if has_rear_units == 'Yes':
            h_left += list(map(lambda x:x+num_units_front, h_left))
            h_mid += list(map(lambda x:x+num_units_front, h_mid))
            h_right += list(map(lambda x:x+num_units_front, h_right))
            lv_bottom += list(map(lambda x:x+num_units_front, lv_bottom))
            lv_middle += list(map(lambda x:x+num_units_front, lv_middle))
            lv_top += list(map(lambda x:x+num_units_front, lv_top))

    floor_unit = 1
   
    for unit in range(1,units+1):     
        bstock.loc[unit] = bldg
        bstock.loc[unit, f'Geometry Building Horizontal Location {not_type}'] = 'None'
        if unit in h_left:
            bstock.loc[unit, f'Geometry Building Horizontal Location {buildtype}'] = 'Left'
        elif unit in h_mid:
            middle_ct += 1
            bstock.loc[unit, f'Geometry Building Horizontal Location {buildtype}'] = 'Middle'
        elif unit in h_right:
            bstock.loc[unit, f'Geometry Building Horizontal Location {buildtype}'] = 'Right'

        if buildtype == 'MF':
            if unit in lv_bottom:
                bstock.loc[unit, 'Geometry Building Level MF'] = 'Bottom'
            elif unit in lv_middle:
                bstock.loc[unit, 'Geometry Building Level MF'] = 'Middle'
            elif unit in lv_top:
                bstock.loc[unit, 'Geometry Building Level MF'] = 'Top'

            if unit <= units/2:
                bstock.loc[unit, 'Orientation'] = front_orient
            else:
                bstock.loc[unit, 'Orientation'] = rear_orient
        else:
            bstock.loc[unit, 'Geometry Building Level MF'] = 'None'
            
    bstock.loc[unit, 'Geometry Building Has Rear Units MF'] = has_rear_units
    bstock['building_code'] = 'bldg_{}'.format(idx)
    bstock['middle_multiplier'] = mid_mult
    
    if master_df.empty:
        master_df = bstock.copy()
    else:
        master_df = master_df.append(bstock)
    
master_df.index= range(1,len(master_df)+1)
master_df.index.rename('Building', inplace=True)

In [7]:
master_df['Geometry Building Horizontal Location SFA'].unique()

array(['Left', 'Middle', 'Right'], dtype=object)

In [8]:
master_df['Geometry Building Number Units SFA'].unique()

array([24, 6, 8, 5, 15, 7, 12, 16, 36, 10, 20, 50, 30], dtype=object)

In [9]:
bldg_map = pd.DataFrame(columns=['building_range', 'mid_multiplier'], index=master_df.building_code.unique())
for bldg in master_df.building_code.unique():
    idx = list(master_df[master_df.building_code == bldg].index.values)
    bldg_map.loc[bldg]['building_range'] = '"{}"'.format(str(idx).replace(' ','').strip(']['))
    bldg_map.loc[bldg]['mid_multiplier'] = master_df[master_df.building_code == bldg].iloc[0].middle_multiplier
bldg_map.head()
bldg_map.to_csv(f'./building_id_mapping_{buildtype}_10000.csv')

In [8]:
master_df

,Usage Level,Corridor,Eaves,Door Area,Overhangs,Interior Shading,Doors,Hot Water Fixtures,Solar Hot Water,Ceiling Fan,...,Misc Freezer,Misc Hot Tub Spa,Misc Pool,Misc Pool Heater,Misc Pool Pump,Ducts,Infiltration,Geometry Perimeter Footprint Ratio,building_code,middle_multiplier
Building,,,,,,,,,,,,,,,,,,,,,
1,Low,Double-Loaded Interior,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,50% Usage,None,Standard Efficiency,...,None,None,None,None,None,"30% Leakage, Uninsulated",40 ACH50,0.13,bldg_1,1
2,Low,Double-Loaded Interior,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,50% Usage,None,Standard Efficiency,...,None,None,None,None,None,"30% Leakage, Uninsulated",40 ACH50,0.13,bldg_1,1
3,Low,Double-Loaded Interior,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,50% Usage,None,Standard Efficiency,...,None,None,None,None,None,"30% Leakage, Uninsulated",40 ACH50,0.13,bldg_1,1
4,Low,Double-Loaded Interior,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,50% Usage,None,Standard Efficiency,...,None,None,None,None,None,"30% Leakage, Uninsulated",40 ACH50,0.13,bldg_1,1
5,Low,Double-Loaded Interior,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,50% Usage,None,Standard Efficiency,...,None,None,None,None,None,"30% Leakage, Uninsulated",40 ACH50,0.13,bldg_1,1
6,Low,Double-Loaded Interior,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,50% Usage,None,Standard Efficiency,...,None,None,None,None,None,"30% Leakage, Uninsulated",40 ACH50,0.13,bldg_1,1
7,Low,Double-Loaded Interior,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,50% Usage,None,Standard Efficiency,...,None,None,None,None,None,"30% Leakage, Uninsulated",40 ACH50,0.13,bldg_1,1
8,Low,Double-Loaded Interior,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,50% Usage,None,Standard Efficiency,...,None,None,None,None,None,"30% Leakage, Uninsulated",40 ACH50,0.13,bldg_1,1
9,Low,Double-Loaded Interior,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,50% Usage,None,Standard Efficiency,...,None,None,None,None,None,"30% Leakage, Uninsulated",40 ACH50,0.13,bldg_1,1


In [9]:
bldg_map

,building_range,mid_multiplier
bldg_1,"""1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,...",1
bldg_2,"""55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,...",1
bldg_3,"""109,110,111,112""",1
bldg_4,"""113,114,115,116,117,118,119,120,121""",1
bldg_5,"""122,123""",1
bldg_6,"""124,125,126,127,128,129,130,131,132,133,134,1...",1
bldg_7,"""178,179,180,181,182,183,184,185,186,187,188,1...",1
bldg_8,"""232,233,234,235,236,237,238,239,240,241,242,2...",1
bldg_9,"""286,287""",1
bldg_10,"""288,289,290""",1


In [10]:
master_df_save = master_df.drop(['building_code', 'middle_multiplier'], axis=1)
master_df_save.to_csv(f'buildstock_{buildtype}_single_10000.csv')

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('buildstock_MF_single_10000.csv')

In [4]:
len(df)

161672

In [6]:
maps = pd.read_csv('building_id_mapping_MF_10000.csv')

In [7]:
maps.head()

,Unnamed: 0,building_range,mid_multiplier
0,bldg_1,"""1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,...",1
1,bldg_2,"""55,56,57,58,59,60""",1
2,bldg_3,"""61,62,63,64,65,66,67,68,69,70,71,72""",1
3,bldg_4,"""73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,...",1
4,bldg_5,"""115,116,117,118,119,120""",1


In [17]:
len(master_df_save)

30000